1. 基于第五课的课件，添加一些条件；
2. 涨跌停限制；
3. 止损
4. 233， 貌似就这些，感觉还是很儿戏啊。。。

1. 首先是跑通第五课的代码；
2. 求涨跌停价格并保存到不复权的表中；
3. 有了这些条件，然后再在回测框架中添加涨跌停限制和止损

先看看目前的进度

查看有哪些database，自己想看的database下面有哪些collection；

```mongodb
show dbs

use quant_01

show collection
```

需要看下，比如daily和daily_hfq有哪些字段，分别有多少条数据

daily有的字段是

code date index ohlc volume

daily_hfq
亦是一样的

也就是说没有做除权因子等操作，也没有记录pre_close

也没有补全信息, and so on

当然也没有做涨跌停的价格录入

回测代码中也有不少需要加索引的地方

先梳理出执行顺序

再确认各个文件里需要index的地方，不光是爬取数据的地方，回测的地方也需要index

database.py / stock_util.py --> daily_crawler.py --> 

basic_crawler.py --> daily_fixing.py -->

compute_high_low_limit.py -->

finance_report.py --> pe_computing.py -->

stock_pool_strategy.py --> backtest.py

```
可以同时运行的有哪些数据呢

daily只是ohlc基本数据

basic_crawler是获取basic的数据

daily_fixing是补充is_trading字段、停牌时的数据、复权因子 和 pre_close

compute_high_low_limit是为了计算涨跌停价格，所以需要前面的都完成以后才能计算；

finance_report是一个独立的过程，可以与其他的过程同时进行

pe_computing需要daily中的close和report的eps，在daily数据和finance_report数据之后才能计算，但是不知道别的数据库在更新daily的时候，会不会影响效率

233，pe的计算果然够慢，如果可能，需要尽快进行哦

```

```
【编程题】为了让回测更接近于实际情况，需要在第5课中回测流程中引入以下逻辑：

1、引入涨停和跌停的判断，跌停时无法卖出，涨停时无法买入；

2、引入止损的风险控制，如果单日亏损超过3%或者累积亏损超过10%则第二天开盘价卖出。 

要求编写完整的可执行代码。自行选定回测周期，附回测结果图。

```

获取股票基本信息，使用新的接口
tushare pro 哦
可能会造成各种不适应

In [1]:
import QUANTAXIS as QA

In [2]:
df = QA.QA_fetch_get_stock_min('tdx', '300257', '2015-01-01', '2018-10-26')

QUANTAXIS>> Selecting the Best Server IP of TDX


USING DEFAULT STOCK IP
USING DEFAULT FUTURE IP


QUANTAXIS>> === The BEST SERVER ===
 stock_ip 60.191.117.167 future_ip 202.103.36.71


In [4]:
import tushare as ts

In [10]:
print(ts.__version__)

1.2.12


In [5]:
token = 'd5deeb2574ff968fbf7cfc89a379a1a2597b2ab6dc56989053ec32a9'
ts.set_token(token)

In [6]:
pro = ts.pro_api()

In [7]:
df = pro.trade_cal(exchange_id='', start_date='20180901', end_date='20181001', fields='exchange_id,cal_date,is_open,pretrade_date', is_open='0')

In [13]:
trade_dates = QA.trade_date_sse

In [16]:
QA.QAUtil.*date*?

In [19]:
trade_range = QA.QA_util_get_trade_range('2015-01-01', '2017-12-31')

In [20]:
ts.get_stock_basics('2015-01-01')

In [22]:
for _date in trade_range:
    if ts.get_stock_basics(_date) is not None:
        print(_date)
        break

2016-08-09


In [23]:
QA.QA_util_get_last_day('2016-08-09')

'2016-08-08'

针对backtest进行debug，看到底是哪里

重构涨跌停计算方式

In [ ]:
df = ts.get_stock_basics()

In [40]:
df.head()

,name,industry,area,pe,outstanding,totals,totalAssets,liquidAssets,fixedAssets,reserved,...,bvps,pb,timeToMarket,undp,perundp,rev,profit,gpr,npr,holders
code,,,,,,,,,,,,,,,,,,,,,
002939,N长城,证券,深圳,44.14,3.10,31.03,5252933.00,0.00,16134.07,767217.75,...,5.25,1.73,20181026,242751.80,0.78,0.00,0.00,0.00,23.52,555970.0
000068,华控赛格,元器件,深圳,0.00,10.07,10.07,322020.63,121044.25,17267.07,119316.82,...,0.62,4.17,19970611,-173199.06,-1.72,-6.87,-88.95,36.36,-34.92,31608.0
300083,劲胜智能,元器件,广东,80.28,11.89,14.32,1152387.88,743815.13,166480.66,366826.81,...,3.93,0.74,20100520,46661.46,0.33,-4.70,-91.40,18.19,0.86,44942.0
002076,雪 莱 特,家用电器,广东,0.00,4.45,7.78,236576.80,141012.47,26288.91,35599.85,...,1.63,2.06,20061025,11410.11,0.15,-23.74,-281.75,22.54,-10.28,51163.0
600758,红阳能源,煤炭开采,辽宁,50.67,6.76,13.31,1634608.75,713384.81,688764.81,340929.41,...,4.08,0.88,19961029,48556.18,0.36,-7.73,-89.50,16.04,1.28,67730.0


In [24]:
start = '2015-01-01'
end = '2017-12-31'

In [25]:
codes = ts.get_stock_basics().index.tolist()

In [26]:
codes[:10]

['002939',
 '000068',
 '300083',
 '002076',
 '600758',
 '002708',
 '600173',
 '601162',
 '000622',
 '002377']

In [35]:
code = codes[1000]

In [36]:
code

'600353'

In [28]:
from database import DB_CONN

In [42]:
sample = np.random.choice(codes, 10)

In [56]:
sample = np.random.choice(codes, 20).tolist()

In [71]:
sample

['600781',
 '002110',
 '600183',
 '002568',
 '603214',
 '603345',
 '000506',
 '300517',
 '603326',
 '002159',
 '603555',
 '600439',
 '002464',
 '002545',
 '002310',
 '002409',
 '002637',
 '002805',
 '002843',
 '002553']

如果初始日就是

In [83]:
st_mark = ['st', 'ST', '*st', '*ST']


for i,code in enumerate(sample):
    try:
        timeToMarket = DB_CONN['basic'].find_one({'code':code}, 
                projection={'code':True, 'timeToMarket':True, '_id':False})['timeToMarket']
    except:
        continue

    daily_cursor = DB_CONN['daily'].find(
            {'code':code, 'date':{'$lte': end, '$gte': timeToMarket}, 'index':False},
            projection={'code':True, 'date':True, 'pre_close':True, '_id':False})

    for daily in daily_cursor:
        date = daily['date']
        
        try:
            pre_close = daily['pre_close']
        except:
            if (date == '2015-01-05') & (timeToMarket != date):
                print('code: %s, time: %s, 数据初始日没有pre_close' % (code, date))
            elif timeToMarket == date:
                issueprice = DB_CONN['basic'].find_one({'code':code},
                          projection={'issueprice':True, '_id':False})['issueprice']

                high_limit = np.round(np.round(issueprice * 1.2, 2) * 1.2, 2)
                low_limit = np.round(np.round(issueprice * 0.8, 2) * 0.8, 2)
            else:
                print('code: %s, time: %s, ipo_date: %s， 请速查原因' % (code, date, timeToMarket))
            continue
        
        
#         print('code: %s, date: %s' % (code, date))
        if date < '2016-08-09':
            _date = '2016-08-09'
        else:
            _date = date
        try:
            name = DB_CONN['basic'].find_one({'code':code, 'date':_date},
                          projection={'name':True, '_id':False})['name']
            last_name = name
        except:
            if i == 0:
                name = DB_CONN['basic'].find_one({'code':code},
                  projection={'name':True, '_id':False})['name']
                last_name = name
            else:
                name = last_name

#         if timeToMarket == date:
#             issueprice = DB_CONN['basic'].find_one({'code':code},
#                       projection={'issueprice':True, '_id':False})['issueprice']

#             high_limit = np.round(np.round(issueprice * 1.2, 2) * 1.2, 2)
#             low_limit = np.round(np.round(issueprice * 0.8, 2) * 0.8, 2)

        if (name[:2] in st_mark) or (name[:3] in st_mark) :
            high_limit = np.round(pre_close * 1.05, 2)
            low_limit = np.round(pre_close * 0.95, 2)

        else:
            high_limit = np.round(pre_close * 1.1, 2)
            low_limit = np.round(pre_close * 0.9, 2)
    print('stock: %s high low limit complish' % code)

code: 600781, time: 2015-01-05, 数据初始日没有pre_close
stock: 600781 high low limit complish
code: 002110, time: 2015-01-05, 数据初始日没有pre_close
stock: 002110 high low limit complish
code: 600183, time: 2015-01-05, 数据初始日没有pre_close
stock: 600183 high low limit complish
code: 002568, time: 2015-01-05, 数据初始日没有pre_close
stock: 002568 high low limit complish
stock: 603345 high low limit complish
code: 000506, time: 2015-02-16, ipo_date: 1993-03-12， 请速查原因
stock: 000506 high low limit complish
stock: 300517 high low limit complish
stock: 603326 high low limit complish
code: 002159, time: 2015-01-05, 数据初始日没有pre_close
stock: 002159 high low limit complish
code: 603555, time: 2015-01-05, 数据初始日没有pre_close
stock: 603555 high low limit complish
code: 600439, time: 2015-01-05, 数据初始日没有pre_close
stock: 600439 high low limit complish
code: 002464, time: 2015-01-19, ipo_date: 2010-08-31， 请速查原因
stock: 002464 high low limit complish
code: 002545, time: 2015-01-05, 数据初始日没有pre_close
stock: 002545 high low limit com

这个bug的原因是基础数据获取的不够完整,不是每一个交易日都有数据

In [79]:
'603214' in df.index

Index(['002939', '000068', '300083', '002076', '600758', '002708', '600173',
       '601162', '000622', '002377',
       ...
       '603386', '600298', '603579', '002942', '002941', '002070', '000693',
       '600680', '600401', '300674'],
      dtype='object', name='code', length=3559)

In [81]:
df.loc['603214',:]

name                     爱婴室
industry                  百货
area                      上海
pe                     41.32
outstanding             0.25
totals                     1
totalAssets           136925
liquidAssets          119606
fixedAssets          6993.16
reserved               41564
reservedPerShare        4.16
esp                     0.72
bvps                    8.31
pb                      4.35
timeToMarket        20180330
undp                 30020.1
perundp                    3
rev                        0
profit                     0
gpr                    27.32
npr                     4.32
holders                19012
Name: 603214, dtype: object

In [59]:
_date

'2016-08-10'

In [60]:
code

'600781'

In [58]:
DB_CONN['basic'].find_one({'code':code, 'date':_date},
     projection={'name':True, '_id':False})['name']

TypeError: 'NoneType' object is not subscriptable

In [52]:
test_cursor = DB_CONN['basic'].find({'code':code},
                      projection={'name':True, 'date':True, '_id':False})

In [53]:
result = [i for i in test_cursor]

In [54]:
result[:10]

[{'date': '2016-08-09', 'name': '通威股份'},
 {'date': '2016-08-11', 'name': '通威股份'},
 {'date': '2016-08-15', 'name': '通威股份'},
 {'date': '2016-08-16', 'name': '通威股份'},
 {'date': '2016-08-17', 'name': '通威股份'},
 {'date': '2016-08-18', 'name': '通威股份'},
 {'date': '2016-08-22', 'name': '通威股份'},
 {'date': '2016-08-23', 'name': '通威股份'},
 {'date': '2016-08-24', 'name': '通威股份'},
 {'date': '2016-08-25', 'name': '通威股份'}]

In [73]:
# test_cursor = DB_CONN['basic'].find({'code':code},
#                       projection={'name':True, 'date':True, '_id':False})
# result = [i for i in test_cursor]
# result

In [69]:
DB_CONN['basic'].find_one({'code':code},
                      projection={'name':True, 'date':True, '_id':False})

{'date': '2016-08-09', 'name': '辅仁药业'}

In [72]:
# daily_cursor = DB_CONN['daily'].find(
#             {'code':code, 'date':{'$lte': end, '$gte': timeToMarket}, 'index':False},
#             projection={'code':True, 'date':True, 'pre_close':True, '_id':False})
# dailies = [i for i in daily_cursor]
# dailies